![title](../img/conv.png)

In [1]:
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

/Users/Supra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# essential network Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 784 # mnist data input (shape: 28*28)
num_classes = 10 # mnist total classes (0-9 digits)
dropout = 0.25 # dropout prob

In [8]:
"""
new AUTO_REUSE option does the trick.
from the tf.variable_scope API docs: if reuse=tf.AUTO_REUSE, 
we create variables if they do not exist, and return them otherwise.
basic example of sharing a variable AUTO_REUSE:
"""

def foo():
  with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
    v = tf.get_variable("v", [1])
  return v

v1 = foo()  # Creates v.
v2 = foo()  # Gets the same, existing v.
assert v1 == v2
"""
more on scope:
https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow
"""

In [10]:
def conv_net_2d(inputs, labels, dropout_prob, reuse, is_training):
    # defining scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # as tf estimator accepts inputs in a dict
        x = inputs['images']
        # mnist images have flat 784 features, conv net takes 4d inputs
        x = tf.reshape(x, shape=[-1,28,28,1])
        # first conv layer with 32 filters and 5x5 kernel
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # max pool layer with strides=2 and kernel_size=2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        # second conv layer with 64 filters and 3x3 kernel
        conv2 = tf.layers.conv2d(x, 64, 3, activation=tf.nn.relu)
        # max pool layer with strides=2 and kernel_size=2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        # flatten the image
        fc1 = tf.contrib.layers.flatten(conv2)
        fc1 = tf.layers.dense(fc1, 1024)
        # if is_training is False then dropout won't be applied
        fc1 = tf.layers.dropout(fc1, rate=dropout_prob, training=is_training)
        out = tf.layers.dense(fc1, labels)
        return out